In [1]:
from bs4 import BeautifulSoup
import requests
import time
import csv
from tqdm import tqdm

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',  # Identifie le navigateur
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',  # Accepte plusieurs types de contenu
    'Accept-Encoding': 'gzip, deflate, br',  # Gère la compression des données
    'Accept-Language': 'en-US,en;q=0.9,fr;q=0.8',  # Précise les langues acceptées
    'Connection': 'keep-alive',  # Maintient la connexion ouverte pour des requêtes multiples
    'Referer': 'https://www.huffingtonpost.fr/agriculture/',  # URL référente pour simuler une navigation depuis la page d'accueil
    'Origin': 'https://www.huffingtonpost.fr',  # L'origine de la requête pour plus de légitimité
    'Cache-Control': 'max-age=0',  # Assure que le cache ne soit pas utilisé pour cette requête
}


In [3]:
def scrape_all_page(url):
    all_articles = []  # Liste des articles collectés
    seen_articles = set()  # Ensemble pour éviter les doublons d'URL

    try:
        # Envoyer une requête HTTP pour récupérer la page
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Trouver tous les articles via la classe "articlePreview-content"
            articles = soup.find_all("div", class_="articlePreview-content")
            
            for article in articles:
                # Extraire le lien de l'article
                link_element = article.find("a", href=True)
                article_url = f"https://www.huffingtonpost.fr{link_element['href']}" if link_element else ""

                # Extraire le titre
                title_element = article.find("div", class_="title")
                title_text = title_element.find("h2").get_text(strip=True) if title_element else ""

                # Extraire la description
                description_element = article.find("div", class_="articlePreview-chapo")
                description_text = description_element.get_text(strip=True) if description_element else ""

                # Vérifier si l'article est déjà collecté
                if article_url and article_url not in seen_articles:
                    all_articles.append([title_text, description_text, article_url])
                    seen_articles.add(article_url)

            return all_articles

        else:
            print(f"Erreur HTTP {response.status_code} lors de l'accès à {url}")
            return []

    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return []


In [4]:
def scrape_all_pages(base_url, total_pages):
    all_articles = []  # Liste pour collecter tous les articles

    for page_num in range(1, total_pages + 1):
        # Construire l'URL pour chaque page
        if page_num == 1:
            url = base_url  # La première page n'a pas de suffixe "?page="
        else:
            url = f"{base_url}?page={page_num}"
        
        print(f"Scraping de la page : {url}")
        articles = scrape_all_page(url)  # Appeler la fonction pour une page
        all_articles.extend(articles)  # Ajouter les articles collectés

        print(f"Page {page_num} : {len(articles)} articles collectés.")

    print(f"Scraping terminé : {len(all_articles)} articles collectés au total.")
    return all_articles


In [5]:
# Fonction pour exporter les résultats en CSV
def export_to_csv_2(dataset, filename="filtered_articles.csv"):
    with open(filename, mode='w', encoding='utf-8', newline='') as file:
        fieldnames = ["Title", "Summary", "Date", "Paragraphs"]
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for data in dataset:
            # Convertir la liste de paragraphes en une seule chaîne avec des doubles sauts de ligne
            data["Paragraphs"] = "\n\n".join(data["Paragraphs"])
            writer.writerow(data)

    print(f"Les résultats ont été sauvegardés dans '{filename}'.")

def export_to_csv(dataset, filename="articles_dataset.csv"):
    # Ouvrir le fichier CSV en mode écriture
    with open(filename, mode='w', encoding='utf-8', newline='') as file:
        # Définir les noms des colonnes (en-têtes)
        fieldnames = ["Title", "Summary", "URL"]
        
        # Créer un objet DictWriter pour écrire dans le fichier
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        
        # Écrire les en-têtes dans le fichier
        writer.writeheader()
        
        # Écrire chaque ligne de données dans le fichier
        for data in dataset:
            writer.writerow(data)

    print(f"Les résultats ont été sauvegardés dans '{filename}'.")


In [10]:
base_url = "https://www.huffingtonpost.fr/agriculture/"
total_pages = 37

# Scraper tous les articles des 37 pages
all_articles = scrape_all_pages(base_url, total_pages)

# Sauvegarder les résultats dans un fichier CSV
export_to_csv([{"Title": title, "Summary": desc, "URL": url} for title, desc, url in all_articles], "agriculture_huffpost.csv")


Scraping de la page : https://www.huffingtonpost.fr/agriculture/
Erreur HTTP 406 lors de l'accès à https://www.huffingtonpost.fr/agriculture/
Page 1 : 0 articles collectés.
Scraping de la page : https://www.huffingtonpost.fr/agriculture/?page=2
Erreur HTTP 406 lors de l'accès à https://www.huffingtonpost.fr/agriculture/?page=2
Page 2 : 0 articles collectés.
Scraping de la page : https://www.huffingtonpost.fr/agriculture/?page=3
Erreur HTTP 406 lors de l'accès à https://www.huffingtonpost.fr/agriculture/?page=3
Page 3 : 0 articles collectés.
Scraping de la page : https://www.huffingtonpost.fr/agriculture/?page=4
Erreur HTTP 406 lors de l'accès à https://www.huffingtonpost.fr/agriculture/?page=4
Page 4 : 0 articles collectés.
Scraping de la page : https://www.huffingtonpost.fr/agriculture/?page=5
Erreur HTTP 406 lors de l'accès à https://www.huffingtonpost.fr/agriculture/?page=5
Page 5 : 0 articles collectés.
Scraping de la page : https://www.huffingtonpost.fr/agriculture/?page=6
Erreur 

In [7]:
# Fonction pour extraire les informations d'un article
def fetch_article_details(article_data, headers):
    title, summary, url = article_data
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire le titre
            title_tag = soup.find('h1', class_='article__title')
            title = title_tag.get_text(strip=True) if title_tag else "Titre non trouvé"

            # Extraire la description (résumé)
            summary_tag = soup.find('p', class_='article__desc')
            summary = summary_tag.get_text(strip=True) if summary_tag else "Résumé non trouvé"

            # Extraire la date
            date_tag = soup.find("span", class_="article-metas__date")
            date = date_tag.get_text(strip=True) if date_tag else "Date non trouvée"

            # Extraire les paragraphes à partir de la classe 'article__paragraph'
            paragraphs = soup.find_all('p', class_='article__paragraph')
            paragraph_list = [p.get_text(strip=True) for p in paragraphs]

            return {
                "Title": title,
                "Summary": summary,
                "Date": date,
                "Paragraphs": paragraph_list
            }
        else:
            print(f"Erreur HTTP {response.status_code} pour {url}")
            return None
    except requests.RequestException as e:
        print(f"Erreur de requête pour {url}: {e}")
        return None


In [8]:
# Fonction pour vérifier si "bio" est dans le texte
def contains_bio(text):
    return 'bio' in text.lower()

# Fonction pour filtrer les articles contenant le mot "bio"
def filter_articles_with_bio(dataset):
    filtered_articles = []
    total_articles = len(dataset)
    
    # Initialiser tqdm pour la boucle avec un calcul du temps estimé
    start_time = time.time()  # Démarrer le chronomètre
    
    for i, article in tqdm(enumerate(dataset), total=total_articles, desc="Filtrage des articles", unit="article"):
        # Vérifier si le mot "bio" est dans le titre ou le résumé
        if contains_bio(article["Title"]) or contains_bio(article["Summary"]):
            filtered_articles.append(article)
        else:
            # Vérifier chaque paragraphe si le mot "bio" est présent
            for paragraph in article["Paragraphs"]:
                if contains_bio(paragraph):
                    filtered_articles.append(article)
                    break
        
        # Calculer et afficher le temps estimé restant
        elapsed_time = time.time() - start_time  # Temps écoulé
        progress = (i + 1) / total_articles  # Progression
        estimated_time_left = elapsed_time / progress - elapsed_time  # Temps estimé restant
        print(f"\rProgression : {progress * 100:.2f}% - Temps restant estimé : {estimated_time_left / 60:.2f} minutes", end="")
    
    print()  # Pour un retour à la ligne après l'affichage du progrès
    return filtered_articles

In [9]:
# Scraper les détails de chaque article dans `all_articles`
detailed_articles = []

for article_data in all_articles:
    details = fetch_article_details(article_data, headers)
    if details:  # Si les détails sont récupérés avec succès
        detailed_articles.append(details)

# Filtrer les articles contenant le mot "bio"
filtered_articles = filter_articles_with_bio(detailed_articles)

# Exporter les articles filtrés en CSV
export_to_csv_2(filtered_articles, "bio_huffpost.csv")

Filtrage des articles: 0article [00:00, ?article/s]


Les résultats ont été sauvegardés dans 'bio_huffpost.csv'.
